In [1]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
import numpy as np
import scipy

print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Python version:", sys.version)

NumPy version: 1.26.4
SciPy version: 1.13.1
Python version: 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]


In [23]:
n,d,k,tau = 50,12,1,100
W,Lambda = np.ones((d, 1)),1
W,Lambda

(array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]),
 1)

In [24]:
np.random.seed(101) #seed = 41,51,...


z = np.random.normal(0, 1, size=(n, 1))
x0= np.dot(W,np.transpose(z))
norm_of_raw_data = np.linalg.norm(x0,ord=2)
x0.shape,norm_of_raw_data

((12, 50), 26.000079468217375)

In [25]:
# Seed
np.random.seed(450)

mean = np.zeros(d)  
cov = (1/tau) * np.eye(d)  
E = np.random.multivariate_normal(mean, cov, size= n).T
# contaminate 
x = x0+E
x.shape

(12, 50)

In [26]:
import numpy as np

def iterate_k_times(X, n, d, k, tau, nu):
   
    sigma_w = 1.0
    mu_w = 0.1 * np.ones((d, 1))     
    mu_z = 0.1 * np.ones((n, 1))      
    sigma_z = np.ones((n, 1))
    a = np.ones(n) * (nu + d) / 2
    b = np.ones(n)
    
    tr_term = np.zeros((100, 1))
    
    for _ in range(k):
        loc = 0
        # Calculate mu_z
        Xi = np.diag(tau * a * b)
        inv_Xi_mu_z = np.dot(Xi, mu_z)
        sigma_w = 1.0 / (np.dot(mu_z.T, inv_Xi_mu_z) + 1)

        mu_w = np.dot(X, np.dot(inv_Xi_mu_z, sigma_w))

        # Update sigma_z and mu_z for each i
        for i in range(n):
            
            sigma_z[i] = 1.0 / (tau * a[i] * b[i] * (d * sigma_w + np.dot(mu_w.T, mu_w)) + 1)
            mu_z[i] = tau * a[i] * b[i] * sigma_z[i] * np.dot(mu_w.T, X[:, i])

        # Update a and b
        for i in range(n):   
            tr_term[i] = (mu_z[i]**2 + sigma_z[i]) * (d * sigma_w + np.dot(mu_w.T, mu_w))
            b[i] = 1.0 / (((tau) / 2) * ((np.dot(X[:, i].T, X[:, i]) - np.dot(X[:, i].T, np.dot(mu_w, mu_z[i])) 
                        - np.dot(mu_z[i].T, np.dot(mu_w.T, X[:, i])) + tr_term[i].item())) + nu / 2)

    return sigma_w, mu_w, mu_z, sigma_z, a, b


X = x 
n = 50
d = 12
k = 20000
tau = 100.0
nu = 10

sigma_w, mu_w, mu_z, sigma_z, a, b = iterate_k_times(X, n, d, k, tau, nu)

print("Final values after {} iterations:".format(k))
print("a:\n", a)
print("b:\n", b)


Final values after 20000 iterations:
a:
 [11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.
 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.
 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
b:
 [0.08238256 0.07395704 0.11434164 0.05563316 0.11096892 0.05354382
 0.0879406  0.10834879 0.12359097 0.08606148 0.13402164 0.08941699
 0.09819056 0.07999426 0.076482   0.09167261 0.12009129 0.10227734
 0.13839534 0.12092491 0.06584915 0.12162362 0.11846441 0.10841229
 0.10915582 0.13157356 0.09905925 0.07620307 0.16094965 0.07212473
 0.08912551 0.10343836 0.07097541 0.09458452 0.1115445  0.08256384
 0.09635126 0.11780437 0.07419319 0.07702269 0.08454349 0.06347889
 0.08533531 0.05107753 0.10879651 0.07798295 0.1399481  0.08489055
 0.09717384 0.0958548 ]


In [27]:
x_estimated= np.dot(mu_w,np.transpose(mu_z))
np.linalg.norm((x_estimated-x0),ord=2)

0.627197187035614

In [28]:
x_estimated[:,25]

array([0.37082654, 0.37213701, 0.38253212, 0.376618  , 0.37465504,
       0.37951591, 0.37429858, 0.38171413, 0.37436212, 0.37801527,
       0.37430142, 0.37587076])

In [29]:
x0[:,25]

array([0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784, 0.39052784, 0.39052784, 0.39052784,
       0.39052784, 0.39052784])